In [9]:
import cv2
import numpy as np
import os

In [10]:
def fill_color_demo(image):
    copyIma = image.copy()
    h, w = image.shape[:2]
    print(h, w)
    mask = np.zeros([h+2, w+2], np.uint8)
    
    cv2.floodFill(copyIma, mask, (0, 0), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)
    cv2.floodFill(copyIma, mask, (w-1, 0), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  
    cv2.floodFill(copyIma, mask, (0, h-1), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  
    cv2.floodFill(copyIma, mask, (w-1, h-1), (0, 0, 0), (100, 100, 100), (50, 50, 50), cv2.FLOODFILL_FIXED_RANGE)  

    return copyIma

In [11]:
source_path = "../../../Data/jpg/CT_jpg/Original"
result_path = "../../../Data/jpg/CT_jpg/CCL_Results"

source_sparse = "Sparse_KKW_n=2000_d=6.jpg"
source_uniform = "Uniform_KKW_n=2000_d=6.jpg"
target_sparse = "Sparse_KKW_n=2000_d=6_Result_CCL.jpg"
target_uniform = "Uniform_KKW_n=2000_d=6_Result_CCL.jpg"
target_ff_uniform = "Uniform_KKW_n=2000_d=6_FF_CCL.jpg"
target_ff_sparse = "Sparse_KKW_n=2000_d=6_FF_CCL.jpg"
fpath = "n=500d=6_23525.jpg"
# # 读入图片
# img_ff = cv2.imread(os.path.join(source_path, source_sparse))
# img_ff = cv2.imread(os.path.join(source_path, source_uniform))
img = cv2.imread(fpath)
img_ff = fill_color_demo(img)
# img_ff = fill_color_demo(img)
# cv2.imwrite(os.path.join(result_path, target_ff_sparse), img_ff)


2000 2000


In [12]:
# 中值滤波，去噪
img_ff = cv2.medianBlur(img_ff, 3)
gray = cv2.cvtColor(img_ff, cv2.COLOR_BGR2GRAY)
# cv2.imwrite(os.path.join(result_dir, grey_file), gray)

#Calulate the canvas size by the node numbers and radius of node
canvas_size = img_ff.shape[0]
#Calulate the ratio for enlarge the coorodinate of node that fitting the canvas size
area_min_radio = 5000 / 4000
area_min = area_min_radio * canvas_size

# 阈值分割得到二值化图片
ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# cv2.imwrite(os.path.join(result_dir, binary_file), binary)
# 膨胀操作
# kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# bin_clo = cv2.dilate(binary, kernel2, iterations=2)

# 连通域分析
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=4)
# num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_clo, connectivity=8)

# 查看各个返回值
# 连通域数量
print('num_labels = ',num_labels)
# 连通域的信息：对应各个轮廓的x、y、width、height和面积
print('stats = ',stats)
# 连通域的中心点
print('centroids = ',centroids)
# 每一个像素的标签1、2、3.。。，同一个连通域的标签是一致的
print('labels = ',labels)

# 不同的连通域赋予不同的颜色
# output = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
# output[:, :] = [255, 255, 255] 

output = img
for i in range(1, num_labels):
    # Area of the component
    x, y, w, h, area = stats[i]
#     area = stats[i, cv2.CC_STAT_AREA] 
    if area > area_min:
        # cv2.rectangle(output, (x, y), (x + w, y + h), (0, 0, 255), 2)

#         print(str(i) + " area is " + str(area))
        mask = labels == i
        output[:, :, 0][mask] = 255
        output[:, :, 1][mask] = 0
        output[:, :, 2][mask] = 0

#         componentMask = (label_ids == i).astype("uint8") * 255
#         output = cv2.bitwise_or(output, componentMask)
        
# cv2.imwrite(os.path.join(result_path, target_uniform), output)
# cv2.imwrite(os.path.join(result_path, target_sparse), output)
cv2.imwrite("result.jpg", output)


num_labels =  1199
stats =  [[      0       0    2000    2000 2381344]
 [   1354       8      37      42     822]
 [   1351      49      38      42     820]
 ...
 [    733    1917      24      29     391]
 [    660    1921      62      62    2946]
 [    713    1925      37      49     830]]
centroids =  [[1003.97128176 1004.28074146]
 [1376.05596107   32.95742092]
 [1365.65731707   65.01097561]
 ...
 [ 746.79028133 1927.76726343]
 [ 687.55600815 1950.33910387]
 [ 729.54457831 1950.14337349]]
labels =  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


True

In [13]:
def jpg_ccl(binary):
    # 连通域分析
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=4)
    return num_labels, labels, stats, centroids

In [14]:
import cv2
import numpy as np
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

num_labels_list = []
labels_list = []
stats_list = []
centroids_list = []
ccl_dict = {}
for root, dirs, files in os.walk("../../jpg_files", topdown=False):
    for name in files:
        if '.jpg' in name:
            fpath = os.path.join(root, name)
            img = cv2.imread(fpath)
            img = cv2.medianBlur(img, 3)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

            print(fpath)
            num_labels, labels, stats, centroids = jpg_ccl(img)
            temp_list = [num_labels, labels, stats, centroids]
            ccl_dict[fpath] = temp_list
            

In [15]:
ccl_dict

{}